In [2]:
# Import necessary libraries
import sys
sys.path.append('../src')  # Add the src directory to the system path

from data_loader import DataLoader
from data_clean_processing import DataCleanProcessing
import pandas as pd
from scipy import stats
import numpy as np

In [8]:
# Load the data
data_loader = DataLoader(file_path)
data = data_loader.load_data()

In [9]:
# Display basic info and head of the data
data_loader.basic_info()
data_loader.display_head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000098 entries, 0 to 1000097
Data columns (total 52 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   UnderwrittenCoverID       1000098 non-null  int64  
 1   PolicyID                  1000098 non-null  int64  
 2   TransactionMonth          1000098 non-null  object 
 3   IsVATRegistered           1000098 non-null  bool   
 4   Citizenship               1000098 non-null  object 
 5   LegalType                 1000098 non-null  object 
 6   Title                     1000098 non-null  object 
 7   Language                  1000098 non-null  object 
 8   Bank                      854137 non-null   object 
 9   AccountType               959866 non-null   object 
 10  MaritalStatus             991839 non-null   object 
 11  Gender                    990562 non-null   object 
 12  Country                   1000098 non-null  object 
 13  Province                  1

,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0


In [11]:
# Clean the data
data_cleaner = DataCleanProcessing(data)
cleaned_data = data_cleaner.clean_missing_values()
print("Verification of no missing values:", data_cleaner.verify_no_missing_values())


Verification of no missing values: True


In [12]:
# Display the first few rows of the cleaned data
cleaned_data.head()

,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0


In [41]:
# Function to perform chi-squared test
def chi_squared_test(group_a, group_b):
    contingency_table = pd.crosstab(group_a, group_b)
    if contingency_table.size == 0:
        raise ValueError("No data available for chi-squared test.")
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    return chi2, p

In [42]:
# Function to perform t-test
def t_test(group_a, group_b):
    t_stat, p = ttest_ind(group_a, group_b, equal_var=False)
    return t_stat, p

In [44]:
# Function to analyze results
def analyze_results(p_value, hypothesis):
    if p_value < 0.05:
        return f"Reject the null hypothesis for {hypothesis}. There is a significant effect."
    else:
        return f"Fail to reject the null hypothesis for {hypothesis}. No significant effect detected."

In [27]:
print(cleaned_data.columns)

Index(['UnderwrittenCoverID', 'PolicyID', 'TransactionMonth',
       'IsVATRegistered', 'Citizenship', 'LegalType', 'Title', 'Language',
       'Bank', 'AccountType', 'MaritalStatus', 'Gender', 'Country', 'Province',
       'PostalCode', 'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'mmcode',
       'VehicleType', 'RegistrationYear', 'make', 'Model', 'Cylinders',
       'cubiccapacity', 'kilowatts', 'bodytype', 'NumberOfDoors',
       'VehicleIntroDate', 'AlarmImmobiliser', 'TrackingDevice',
       'CapitalOutstanding', 'NewVehicle', 'SumInsured', 'TermFrequency',
       'CalculatedPremiumPerTerm', 'ExcessSelected', 'CoverCategory',
       'CoverType', 'CoverGroup', 'Section', 'Product', 'StatutoryClass',
       'StatutoryRiskType', 'TotalPremium', 'TotalClaims'],
      dtype='object')


In [45]:
# Cell 4: Hypothesis 1 - Risk differences across provinces
# Assuming 'Province' and 'StatutoryRiskType' columns are relevant for this test
try:
    chi2, p = chi_squared_test(cleaned_data['Province'], cleaned_data['StatutoryRiskType'])
    print(f"Hypothesis 1: Chi-squared test statistic: {chi2}, p-value: {p}")
    print(analyze_results(p, "Risk differences across provinces"))
except ValueError as e:
    print(f"Hypothesis 1: {e}")

Hypothesis 1: Chi-squared test statistic: 0.0, p-value: 1.0
Fail to reject the null hypothesis for Risk differences across provinces. No significant effect detected.


In [36]:
# Assuming 'PostalCode' and 'StatutoryRiskType' columns are relevant for this test
chi2, p = chi_squared_test(cleaned_data['PostalCode'], cleaned_data['StatutoryRiskType'])
print(f"Hypothesis 2: Chi-squared test statistic: {chi2}, p-value: {p}")
print(analyze_results(p, "Risk differences between zipcodes"))

Hypothesis 2: Chi-squared test statistic: 0.0, p-value: 1.0
Fail to reject the null hypothesis for Risk differences between zipcodes. No significant effect detected.


In [39]:
# Assuming 'PostalCode' and 'TotalPremium' - 'TotalClaims' columns are relevant for this test
cleaned_data['Margin'] = cleaned_data['TotalPremium'] - cleaned_data['TotalClaims']
zipcodes = cleaned_data['PostalCode'].unique()
if len(zipcodes) &gt; 1:
    group_a = cleaned_data[cleaned_data['PostalCode'] == zipcodes[0]]['Margin']
    group_b = cleaned_data[cleaned_data['PostalCode'] == zipcodes[1]]['Margin']
    t_stat, p = t_test(group_a, group_b)
    print(f"Hypothesis 3: t-test statistic: {t_stat}, p-value: {p}")
    print(analyze_results(p, "Margin differences between zipcodes"))

SyntaxError: invalid syntax (1421896457.py, line 4)

In [40]:
# Assuming 'Gender' and 'StatutoryRiskType' columns are relevant for this test
group_a = cleaned_data[cleaned_data['Gender'] == 'Female']['StatutoryRiskType']
group_b = cleaned_data[cleaned_data['Gender'] == 'Male']['StatutoryRiskType']
chi2, p = chi_squared_test(group_a, group_b)
print(f"Hypothesis 4: Chi-squared test statistic: {chi2}, p-value: {p}")
print(analyze_results(p, "Risk differences between Women and Men"))

ValueError: No data; `observed` has size 0.

In [ ]:
# Hypothesis 3: Margin differences between zipcodes
print("\nHypothesis 3: Margin differences between zipcodes")
zipcode_margins = data.groupby('zipcode')['margin'].mean()
group_a = data[data['zipcode'] == 'zip_a']['margin']
group_b = data[data['zipcode'] == 'zip_b']['margin']
t_stat, p = t_test(group_a, group_b)
print(f"T-test statistic: {t_stat}, p-value: {p}")

In [ ]:
# Hypothesis 4: Risk differences between Women and Men
print("\nHypothesis 4: Risk differences between Women and Men")
group_a = data[data['gender'] == 'Female']['risk']
group_b = data[data['gender'] == 'Male']['risk']
t_stat, p = t_test(group_a, group_b)
print(f"T-test statistic: {t_stat}, p-value: {p}")

In [ ]:
# Analyze and Report
def analyze_results(p_value, hypothesis):
    if p_value < 0.05:
        print(f"Reject the null hypothesis for {hypothesis}. There is a significant effect.")
    else:
        print(f"Fail to reject the null hypothesis for {hypothesis}. No significant effect detected.")

analyze_results(p, "Hypothesis 1: Risk differences across provinces")
analyze_results(p, "Hypothesis 2: Risk differences between zipcodes")
analyze_results(p, "Hypothesis 3: Margin differences between zipcodes")
analyze_results(p, "Hypothesis 4: Risk differences between Women and Men")